# Basic stats on the evolution of COVID-19 hospitalizations in departments

In [1]:
%load_ext lab_black
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt
from datetime import timedelta, date
import pandas as pd
import warnings

warnings.filterwarnings("ignore")
pd.plotting.register_matplotlib_converters()

In [3]:
covid_url = (
    "https://www.data.gouv.fr/fr/datasets/r/63352e38-d353-4b54-bfd1-f1b3ee1cabd7"
)
filter_dep = ["971", "972", "973", "974", "976"]  # only metropolitan France
figsize = (15, 10)

#### <br> Load table with COVID-19 data (figures by dates)

In [4]:
df = pd.read_csv(covid_url, sep=";", parse_dates=True, index_col=2)
df = df.query("sexe == 0")  # sum male/female
df.drop(columns=["sexe"], inplace=True)
df.query("dep not in @filter_dep", inplace=True)
df.sort_index(inplace=True)
df.head(2)

,dep,hosp,rea,rad,dc
jour,,,,,
2020-03-18,01,2,0,1,0
2020-03-18,02,41,10,18,11


In [5]:
last_day = df.index.max()
previous_day = last_day - timedelta(days=1)
print("Evolution from ", previous_day, " to ", last_day)

Evolution from  2020-04-01 00:00:00  to  2020-04-02 00:00:00


#### <br> Build a new dataframe with data from two latest days and the associated evolution

In [6]:
sdf = df.loc[last_day]
sdf.reset_index(inplace=True)
pdf = df.loc[previous_day]
pdf.reset_index(inplace=True)
frame = {"dep": sdf["dep"], "prev_hosp": pdf["hosp"], "last_hosp": sdf["hosp"]}
stats = pd.DataFrame(frame)
stats["diff"] = stats["last_hosp"] - stats["prev_hosp"]
stats["percent"] = stats["diff"] / stats["prev_hosp"] * 100
stats.head()

,dep,prev_hosp,last_hosp,diff,percent
0,01,92,105,13,14.130435
1,02,164,186,22,13.414634
2,03,38,36,-2,-5.263158
3,04,22,25,3,13.636364
4,05,52,59,7,13.461538


#### <br> Departments with highest number of hospitalizations on the latest day

In [7]:
stats.sort_values(by="last_hosp", ascending=False).head()

,dep,prev_hosp,last_hosp,diff,percent
75,75,2633,2838,205,7.785796
92,92,1638,1748,110,6.715507
94,94,1368,1474,106,7.748538
93,93,1173,1208,35,2.983802
69,69,1114,1138,24,2.154399


#### <br> Min, max, mean of evolution between the two latest days (in sum and %)

In [8]:
print(
    "Min: ",
    stats["diff"].min(),
    " Mean: ",
    stats["diff"].mean(),
    " Max: ",
    stats["diff"].max(),
)

Min:  -14  Mean:  16.479166666666668  Max:  205


In [9]:
print(
    "Min %: ",
    stats.percent.min(),
    " Mean %: ",
    stats.percent.mean(),
    " Max %: ",
    stats.percent.max(),
)

Min %:  -15.909090909090908  Mean %:  9.098240582802147  Max %:  75.0


#### <br> List of departments with a drecrease of hospitalizations (last two days)

In [10]:
stats.query("diff < 0")

,dep,prev_hosp,last_hosp,diff,percent
2,03,38,36,-2,-5.263158
6,07,85,83,-2,-2.352941
17,18,40,38,-2,-5.000000
18,19,39,38,-1,-2.564103
24,26,240,229,-11,-4.583333
28,2A,88,74,-14,-15.909091
31,31,216,214,-2,-0.925926
36,36,51,49,-2,-3.921569
49,49,150,144,-6,-4.000000
55,55,162,153,-9,-5.555556


#### <br> Departments with highest evolution of hospitalizations (total number)

In [11]:
stats.sort_values(by="diff", ascending=False).head()

,dep,prev_hosp,last_hosp,diff,percent
75,75,2633,2838,205,7.785796
92,92,1638,1748,110,6.715507
94,94,1368,1474,106,7.748538
59,59,613,703,90,14.681892
91,91,687,758,71,10.334789


#### <br> Departments with highest rate of hospitalizations (%)

In [12]:
stats.sort_values(by="percent", ascending=False).head()

,dep,prev_hosp,last_hosp,diff,percent
14,15,12,21,9,75.000000
46,46,8,13,5,62.500000
29,2B,8,12,4,50.000000
10,11,80,101,21,26.250000
41,41,58,73,15,25.862069


#### <br> Focus on some departments

In [13]:
stats.query("dep=='75'")

,dep,prev_hosp,last_hosp,diff,percent
75,75,2633,2838,205,7.785796


In [14]:
stats.query("dep=='71'")

,dep,prev_hosp,last_hosp,diff,percent
71,71,173,169,-4,-2.312139


In [15]:
stats.query("dep=='31'")

,dep,prev_hosp,last_hosp,diff,percent
31,31,216,214,-2,-0.925926
